<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ Вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

In [2]:
# вставьте сюда параметры подключения


In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

## Юнит 3. Предварительный Анализ Данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies).

In [4]:
# текст запроса
query_3_1 = f'''select count(*) as total_vacancies
                from vacancies v
            '''

In [5]:
# результат запроса
total_vacancies = pd.read_sql_query(query_3_1, connection)
total_vacancies

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/452537431.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_vacancies = pd.read_sql_query(query_3_1, connection)


,total_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers).

In [6]:
# текст запроса
query_3_2 = f'''select count(*) as total_employers
                from employers
            '''

In [7]:
# результат запроса
total_employers = pd.read_sql_query(query_3_2, connection)
total_employers

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/128870259.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_employers = pd.read_sql_query(query_3_2, connection)


,total_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''select count(id) as no_of_areas
                from areas a
            '''

In [9]:
# результат запроса
no_of_areas = pd.read_sql_query(query_3_3, connection)
no_of_areas

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/239293661.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  no_of_areas = pd.read_sql_query(query_3_3, connection)


,no_of_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''select count(id) as industry_type
                from industries i
            '''

In [11]:
# результат запроса
industry_type = pd.read_sql_query(query_3_4, connection)
industry_type

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3102458768.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry_type = pd.read_sql_query(query_3_4, connection)


,industry_type
0,294


***

**Выводы по предварительному анализу данных**:

Набор данных содержит пять таблиц: четыре, из которых содержат информацию о вакансиях (vacancies), работодателях (employers), местоположении регионов (areas), и сфере деятельности (industries); пятая таблица (employers_industries) является связующей между таблицей работодателей и сфер деятельности.

Базовый анализ показывает:
1. суммарное количество вакансий: 49197
   - базовый анализ не доказывает, что нет дубликатов в данных, поэтому итоговое количество может быть и меньше
2. суммарное количество работодателей: 23501
   - без детального анализа многие компании могут быть дочерними компаниями материнской/холдинговой компании
3. суммарное количество регионов: 1362
   - запрос показывает большое разнообразие результатов: могут быть города, поселки, регионы
   - можно сделать предположение, что данные не агрегированы по численности населения
4. суммарное количество сфер деятельности: 294
   - без более детального анализа можно предположить, что некоторые должности могут дублировать друг друга

## Юнит 4. Детальный Анализ Вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''select count(v.id) as cnt,
                       a.name
                from vacancies v
                
                --matching values in both tables
                join areas a
                on v.area_id = a.id
                
                --group by region
                group by a.name
                
                --sort by most to less
                order by cnt DESC
                
                --return top5 results only
                limit 5
            '''

In [13]:
# результат запроса
result4_1 = pd.read_sql_query(query_4_1, connection)
result4_1

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3432431514.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result4_1 = pd.read_sql_query(query_4_1, connection)


,cnt,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''select count(v.id) as cnt
                from vacancies v
                
                --if either field is not empty
                where (v.salary_from is not null
                or v.salary_to is not null)
            '''

In [15]:
# результат запроса
result4_2 = pd.read_sql_query(query_4_2, connection)
result4_2

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/1439396995.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result4_2 = pd.read_sql_query(query_4_2, connection)


,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''select ceil(avg(v.salary_from)) as mean_salary_start,           --use ceil to roundup the value
                       ceil(avg(v.salary_to)) as mean_salary_end
                from vacancies v
            '''

In [17]:
# результат запроса
result4_3 = pd.read_sql_query(query_4_3, connection)
result4_3

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/1094546933.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result4_3 = pd.read_sql_query(query_4_3, connection)


,mean_salary_start,mean_salary_end
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''select v.schedule || ' + ' || v.employment as work_preference,          --concatenate the columns
                       count(v.id) as total_combination
                from vacancies v
                
                group by work_preference
                
                order by total_combination DESC
                
                --return 2nd result only
                offset 1 limit 1
            '''

In [19]:
# результат запроса
result4_4 = pd.read_sql_query(query_4_4, connection)
result4_4

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/568404625.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result4_4 = pd.read_sql_query(query_4_4, connection)


,work_preference,total_combination
0,Удаленная работа + Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''select v.experience,
                       count(v.id) as cnt_of_vacancies
                from vacancies v
                
                group by v.experience
                
                --sort by less to most
                order by cnt_of_vacancies ASC
            '''

In [21]:
# результат запроса
result4_5 = pd.read_sql_query(query_4_5, connection)
result4_5

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3569830517.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result4_5 = pd.read_sql_query(query_4_5, connection)


,experience,cnt_of_vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

**Выводы по детальному анализу вакансий**:

Детальный анализ показывает:
1. Топ-5 результатов по количеству вакансий приходятся на города-миллионники, логическая корреляция, поскольку в крупных городах больше возможностей для разных видов работ.
2. Только у 48.9% (24073) вакансий заполнены поля по зарплате, то есть больше >50% вакансий не имеют информации о потенциальной заработное плате по позиции.
   - одна из причин может заключаться в том, чтобы побудить людей, которые в противном случае не претендовали бы на должность, т. е. люди с более высокой квалификацией могут настаивать на более высокий уровень заработной платы в процессе подачи заявления;
   - другая причина может заключаться в том, чтобы свести к минимуму недовольство среди членов организации, которые могут получать более низкую зарплату на той же должности.
3. Анализ средней зарплаты показывает диапазон от 70т. до 110,5т., но запрос охватывает все регионы, в том числе страны за пределами России - Казахстан, Белоруссия, поэтому необходим дальнейший анализ по городам для учета выбросов, и несоответствий в ожиданиях по заработной плате.
4. Предпочтение отдаётся полному рабочему дню и полному графику, с ростом интереса к дистанционной работе, и это, вероятно, связано с тем, что скорее всего большинство кандидатов в начале своего карьерного пути.
5. Предпочтение стажа работы начинается от 1-3 лет. Около 15% вакансий рекламируют отсутствие опыта, скорее всего, потому что многие кандидаты меняют свою карьеру или только получили высшее образование. Вакансии, требующие более длительного опыта работы (>6 лет), скорее всего, будут рекламироваться внутри компании, поэтому на HeadHunter рекламируется меньше вакансий (2.7%).

## Юнит 5. Анализ Работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''select e.name as employer_name,
                       count(v.id) as no_of_vacancies
                from vacancies v
                
                join employers e
                on v.employer_id = e.id
                
                group by employer_name
                
                order by no_of_vacancies DESC
                
                limit 5
            '''

In [23]:
# результат запроса
result5_1 = pd.read_sql_query(query_5_1, connection)
result5_1

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/1000295000.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result5_1 = pd.read_sql_query(query_5_1, connection)


,employer_name,no_of_vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''select a.name as region_name,
                       count(e.id) as total_employers,
                       count(v.id) as total_vacancies
                from areas a
                
                --matching values from table employers with (potential) null fields if no matches with table areas
                left join employers e
                on a.id = e.area
                
                left join vacancies v
                on a.id = v.area_id
                
                --filter by vacancy field is empty
                where v.id is null
                
                group by region_name
                
                order by total_employers DESC
                
                --return top result
                limit 1
            '''

In [25]:
# результат запроса
result5_2 = pd.read_sql_query(query_5_2, connection)
result5_2

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/286381114.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result5_2 = pd.read_sql_query(query_5_2, connection)


,region_name,total_employers,total_vacancies
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.

In [26]:
# текст запроса
query_5_3 = f'''select e.name,
                       count(distinct a.id) as no_of_regions
                from vacancies v
                
                join areas a
                on v.area_id = a.id
                
                join employers e
                on v.employer_id = e.id
                
                group by e.name
                
                order by no_of_regions DESC
                
                limit 5
            '''

In [27]:
# результат запроса
result5_3 = pd.read_sql_query(query_5_3, connection)
result5_3

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/603015360.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result5_3 = pd.read_sql_query(query_5_3, connection)


,name,no_of_regions
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [28]:
# текст запроса
query_5_4 = f'''select count(e.id) as no_industry
                from employers e
                
                left join employers_industries ei
                on e.id = ei.employer_id
                
                left join industries i
                on ei.industry_id = i.id
                
                --filter by industry field is empty
                where ei.industry_id is null
            '''

In [29]:
# результат запроса
result5_4 = pd.read_sql_query(query_5_4, connection)
result5_4

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3952296371.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result5_4 = pd.read_sql_query(query_5_4, connection)


,no_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [30]:
# текст запроса
query_5_5 = f'''select e.name,
                       count(i.id) as industry_type
                from employers e
                
                left join employers_industries ei
                on e.id = ei.employer_id
                
                left join industries i
                on ei.industry_id = i.id
                
                --filter by industry field is not empty
                where ei.industry_id is not null
                
                group by e.name
                
                --with specific number of industries only
                having count(e.id) = 4
                
                order by e.name ASC
                
                --return 3rd result only
                offset 2 limit 1
            '''

In [31]:
# результат запроса
result5_5 = pd.read_sql_query(query_5_5, connection)
result5_5

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/219870366.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result5_5 = pd.read_sql_query(query_5_5, connection)


,name,industry_type
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.

In [32]:
# текст запроса
query_5_6 = f'''select count(i.id) as preferred_position
                from employers e
                
                left join employers_industries ei
                on e.id = ei.employer_id
                
                left join industries i
                on ei.industry_id = i.id
                
                --filter by industry field is not empty AND specific position
                where ei.industry_id is not null
                and i.name like '%Разработка программного обеспечения%'
            '''

In [33]:
# результат запроса
result5_6 = pd.read_sql_query(query_5_6, connection)
result5_6

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/1520086702.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result5_6 = pd.read_sql_query(query_5_6, connection)


,preferred_position
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8).

In [34]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

#use the get method
response = requests.get(url)

#create object BeautifulSoup using html.parser
page = BeautifulSoup(response.text, 'html.parser')

#find all links on the page
links = page.find('table', 'standard sortable').find('tbody').find_all('a')

#use slicing to find the relevant list of cities
million_cities = [link['title'] for link in links[4:]]

display(million_cities)

['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [35]:
# текст запроса
query_5_7 = f'''(
                select a.name as region_name,
                       count(v.id) as no_of_vacancies
                from vacancies v
                
                left join employers e
                on v.employer_id = e.id
                
                left join areas a
                on v.area_id = a.id
                
                where e.name = 'Яндекс'
                --find the element in a specific list, convert list to tuple to avoid an error
                and a.name in {tuple(million_cities)}
              
                group by region_name
                )
                
                --combine two query results to return the total vacancy count
                union all
                
                (
                select 'total',                                --set column name as a variable
                       count(v.id) as no_of_vacancies
                from vacancies v
                
                left join employers e
                on v.employer_id = e.id
                
                left join areas a
                on v.area_id = a.id
                
                where e.name = 'Яндекс'
                and a.name in {tuple(million_cities)}
                )
                
                --sort both queries by 2nd column
                order by 2 ASC
            '''

In [36]:
# результат запроса
result5_7 = pd.read_sql_query(query_5_7, connection)
result5_7

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/643164009.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result5_7 = pd.read_sql_query(query_5_7, connection)


,region_name,no_of_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

**Выводы по анализу работодателей**:

Выводы по анализу показывает:
1. Общее количество вакансий рекламируют IT (Яндекс, Ростелеком), банковские (Тинькофф, Сбер) и энергетические компании (Газпром Нефть).
2. Поэтому вполне логично, что одни и те же отрасли рекламируют вакансии в большем количестве регионов. На третьем месте также работа, связанная с ремонтом, так как наблюдается дефицит рабочих рук по всей стране.
3. Около 17% вакансий не имеют подробного описания по отрасли: либо из-за неправильного ввода данных, либо из-за того, что работодатели ищут более квалифицированных работников, которые могут уверенно решать различные типы задач.
4. 7.2% всех вакансии состовляет разработчик программного обеспечения; можно предположить, что интерес к этому виду работ растёт.
5. Анализ данных по большим городам показывает, что только 25% всех позиции в Яндексе рекламируются в больших городах, и работу можно найти либо дистанционно, и в небольшом городе.
   - необходим дальнейший анализ других компаний, чтобы увидеть существует ли аналогичная тенденция.

## Юнит 6. Предметный Анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''select count(v.id) as data_positions
                from vacancies v
                
                --set variable to lower case and filter using a like statement to avoid any near misses in results
                where lower(v.name) like '%данн%'
                or lower(v.name) like '%data%'
            '''

In [38]:
# результат запроса
result6_1 = pd.read_sql_query(query_6_1, connection)
result6_1

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/4161228700.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result6_1 = pd.read_sql_query(query_6_1, connection)


,data_positions
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''select count(v.id) as junior_data_roles
                from vacancies v
                
                where (lower(v.name) like '%data scientist%'
                or lower(v.name) like '%data science%'
                or lower(v.name) like '%исследователь данных%'
                
                or (v.name like '%ML%'and v.name not like '%HTML%')
                
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%')
                
                and (lower(v.name) like '%junior%'
                or lower(v.experience) = 'нет опыта'
                or lower(v.employment) = 'стажировка')
            '''

In [40]:
# результат запроса
result6_2 = pd.read_sql_query(query_6_2, connection)
result6_2

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/1449935675.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result6_2 = pd.read_sql_query(query_6_2, connection)


,junior_data_roles
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''select count(v.id) as sql_preferred
                from vacancies v
                
                where (lower(v.name) like '%data scientist%'
                or lower(v.name) like '%data science%'
                or lower(v.name) like '%исследователь данных%'
                
                or (v.name like '%ML%'and v.name not like '%HTML%')
                
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%')
                
                and (lower(v.key_skills) like '%sql%'
                or lower(v.key_skills) like '%postgres%')
            '''

In [42]:
# результат запроса
result6_3 = pd.read_sql_query(query_6_3, connection)
result6_3

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3306808428.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result6_3 = pd.read_sql_query(query_6_3, connection)


,sql_preferred
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_4 = f'''select count(v.id) as python_preferred
                from vacancies v
                
                where (lower(v.name) like '%data scientist%'
                or lower(v.name) like '%data science%'
                or lower(v.name) like '%исследователь данных%'
                
                or (v.name like '%ML%'and v.name not like '%HTML%')
                
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%')
                
                and lower(v.key_skills) like '%python%'
            '''

In [44]:
# результат запроса
result6_4 = pd.read_sql_query(query_6_4, connection)
result6_4

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/220825717.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result6_4 = pd.read_sql_query(query_6_4, connection)


,python_preferred
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
#use replace() function to find any instances of '\t' and change to ''
#use length() function to find the total number of characters
#apply length() on original column value and changed column value to return the number of elements in column value
query_6_5 = f'''select round(avg(length(v.key_skills) - length(replace(v.key_skills, chr(9), ''))) + 1, 2) as average_skills
                from vacancies v
                
                where (lower(v.name) like '%data scientist%'
                or lower(v.name) like '%data science%'
                or lower(v.name) like '%исследователь данных%'
                
                or (v.name like '%ML%'and v.name not like '%HTML%')
                
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%')   
                
                and v.key_skills is not null             
            '''

In [46]:
# результат запроса
result6_5 = pd.read_sql_query(query_6_5, connection)
result6_5

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3098249384.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result6_5 = pd.read_sql_query(query_6_5, connection)


,average_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
#use coalesce to replace null values with another variable
query_6_6 = f'''select v.experience as work_experience,
                       ceil(avg(((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2))) as average_salary
                from vacancies v
                
                where (lower(v.name) like '%data scientist%'
                or lower(v.name) like '%data science%'
                or lower(v.name) like '%исследователь данных%'
                
                or (v.name like '%ML%'and v.name not like '%HTML%')
                
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%')   
                
                and (v.salary_from is not null
                or v.salary_to is not null)
                
                group by work_experience
                
                order by work_experience ASC
            '''

In [48]:
# результат запроса
result6_6 = pd.read_sql_query(query_6_6, connection)
result6_6

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/2920015587.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result6_6 = pd.read_sql_query(query_6_6, connection)


,work_experience,average_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

**Выводы по предметному анализу**:

Предметный анализ показывает:
1. 1771 позиций по анализу данным, то есть около 3,6% от общего числа вакансий, свидетельствуют о медленном росте отрасли.
   - с позициями junior только 0.1%, что говорит о высококонкурентной сфере, в которую сложно проникнуть.
2. Python (351, 0,71%) более популярен, чем SQL (201, 0,4%): знание первого программного языка важнее для сферы анализа данных.
   - наличие навыков, так же очень высоко: в среднем необходимо иметь около 6-7 навыков для работы в анализе данных.
3. Однако, заработная плата выше среднего (30т.) по стране:
   - нет опыта: 75т.;
   - 1-3 года: 140т.;
   - 3-6 лет: 240т.

## Юнит 7. Дополнительный Анализ

1. Средняя зарплата по городам-миллионикам

In [49]:
# текст запроса
query_7_1 = f'''select a.name as region_name,
                       ceil(avg(v.salary_from)) as salary_start,           
                       ceil(avg(v.salary_to)) as salary_end,
                       ceil(avg(((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2))) as average_salary
                from vacancies v
                
                left join areas a
                on v.area_id = a.id
                
                where a.name in {tuple(million_cities)}
              
                group by region_name
                
                order by average_salary DESC
            '''

In [50]:
# результат запроса
result_7_1 = pd.read_sql_query(query_7_1, connection)
result_7_1

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3765338456.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7_1 = pd.read_sql_query(query_7_1, connection)


,region_name,salary_start,salary_end,average_salary
0,Москва,113410.0,160595.0,134141.0
1,Новосибирск,91272.0,137573.0,107777.0
2,Санкт-Петербург,89893.0,127249.0,104410.0
3,Нижний Новгород,83734.0,126921.0,96059.0
4,Пермь,78933.0,130215.0,95985.0
5,Ростов-на-Дону,79392.0,119709.0,94066.0
6,Краснодар,78484.0,117144.0,92329.0
7,Казань,76118.0,119942.0,92208.0
8,Екатеринбург,77489.0,120404.0,91896.0
9,Омск,74899.0,107686.0,86393.0


2. Количество вакансий по городам-миллионикам по большим компаниям в сравнении с общим количеством вакансий

In [51]:
# текст запроса
query_7_2 = f'''with total_vacancies as (select e.name as employer_name,
                                                count(v.id) as vacancies
                                        from vacancies v
                
                                        join employers e
                                        on v.employer_id = e.id
                
                                        group by e.name
                
                                        order by vacancies DESC
                
                                        limit 5)
                                       
            select e.name as employer_name,
                   count(v.id) as no_of_vacancies,
                   tv.vacancies as total_vacancies
            from vacancies v
                
            left join employers e
            on v.employer_id = e.id
                
            left join areas a
            on v.area_id = a.id
              
            left join total_vacancies tv
            on e.name = tv.employer_name
              
            where e.name in ('Ростелеком', 'Тинькофф', 'СБЕР', 'Газпром нефть')
            and a.name in {tuple(million_cities)}
              
            group by e.name, total_vacancies
                
            order by 2 ASC
            '''

In [52]:
# результат запроса
result_7_2 = pd.read_sql_query(query_7_2, connection)
result_7_2

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/2117260267.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7_2 = pd.read_sql_query(query_7_2, connection)


,employer_name,no_of_vacancies,total_vacancies
0,Ростелеком,214,491
1,Газпром нефть,245,331
2,Тинькофф,285,444
3,СБЕР,364,428


3а. Список топ-10 средней заработной платы по регионам

In [53]:
# текст запроса
query_7_3a = f'''select a.name as region,
                       ceil(avg(((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2))) as average_salary
                from vacancies v
                
            left join areas a
            on v.area_id = a.id
              
            group by region
            
            --filter by non-null salary amounts only
            having ceil(avg(((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2))) > 0
            
            order by 2 DESC
            
            --return top 10
            limit 10
            '''

In [54]:
# результат запроса
result_7_3a = pd.read_sql_query(query_7_3a, connection)
result_7_3a

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3939557762.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7_3a = pd.read_sql_query(query_7_3a, connection)


,region,average_salary
0,Германия,452751.0
1,Болгария,324348.0
2,Испания,300000.0
3,Черногория,288590.0
4,Литва,263019.0
5,Польша,257156.0
6,Петропавловка (Республика Бурятия),250000.0
7,Турция,235436.0
8,Кипр,223736.0
9,Кутаиси,220475.0


3б. Список топ-10 средней заработной платы по профессиям, связанных с анализом данных по регионам

In [55]:
# текст запроса
query_7_3b = f'''select a.name as region,
                       ceil(avg(((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2))) as average_salary
                from vacancies v
                
            left join areas a
            on v.area_id = a.id
            
            where (lower(v.name) like '%data scientist%'
            or lower(v.name) like '%data science%'
            or lower(v.name) like '%исследователь данных%'
                
            or (v.name like '%ML%'and v.name not like '%HTML%')
                
            or lower(v.name) like '%machine learning%'
            or lower(v.name) like '%машинн%обучен%')   
            
            group by region
            
            --filter by non-null salary amounts only
            having ceil(avg(((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2))) > 0
            
            order by 2 DESC
            
            --return top 10
            limit 10
            '''

In [56]:
# результат запроса
result_7_3b = pd.read_sql_query(query_7_3b, connection)
result_7_3b

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/311879772.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7_3b = pd.read_sql_query(query_7_3b, connection)


,region,average_salary
0,Кипр,300000.0
1,Армения,268863.0
2,Черногория,233794.0
3,Турция,233794.0
4,Сербия,233794.0
5,Москва,207630.0
6,Белгород,200000.0
7,Санкт-Петербург,173336.0
8,Новосибирск,166334.0
9,Рязань,136667.0


4а. Топ-10 отраслей по средней зарплате

In [57]:
# текст запроса
query_7_4a = f'''select i.name as profession,
                       ceil(avg(((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2))) as average_salary
                from vacancies v
                
                left join employers e
                on v.employer_id = e.id
                
                left join employers_industries ei
                on e.id = ei.employer_id
                
                left join industries i
                on ei.industry_id = i.id
                
                where ei.industry_id is not null
                
                and (v.salary_from is not null
                or v.salary_to is not null)
                
                group by profession
                
                order by 2 DESC
                
                limit 10
            '''

In [58]:
# результат запроса
result_7_4a = pd.read_sql_query(query_7_4a, connection)
result_7_4a

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3549463726.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7_4a = pd.read_sql_query(query_7_4a, connection)


,profession,average_salary
0,"Агрохимия (продвижение, оптовая торговля)",381429.0
1,Агрохимия (производство),339875.0
2,Игорный бизнес,191905.0
3,Корма для животных (производство),168300.0
4,Зоомагазин,167613.0
5,Кадровые агентства,144530.0
6,"Лакокрасочная продукция, сырье (производство)",142803.0
7,"Бытовая химия, парфюмерия, косметика (произво...",130120.0
8,"Лакокрасочная продукция, сырье (продвижение, о...",128445.0
9,"Дорожно-строительная техника, сельскохозяйстве...",122878.0


4б. Топ-10 отраслей по средней зарплате по позициям, связанным с анализом данных

In [59]:
# текст запроса
query_7_4b = f'''select i.name as profession,
                       ceil(avg(((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2))) as average_salary
                from vacancies v
                
                left join employers e
                on v.employer_id = e.id
                
                left join employers_industries ei
                on e.id = ei.employer_id
                
                left join industries i
                on ei.industry_id = i.id
                
                where ei.industry_id is not null
                
                and (v.salary_from is not null
                or v.salary_to is not null)
                
                and (lower(v.name) like '%data scientist%'
                or lower(v.name) like '%data science%'
                or lower(v.name) like '%исследователь данных%'
                
                or (v.name like '%ML%'and v.name not like '%HTML%')
                
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%')
                
                group by profession
                
                order by 2 DESC
                
                limit 10
            '''

In [60]:
# результат запроса
result_7_4b = pd.read_sql_query(query_7_4b, connection)
result_7_4b

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/3360556187.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7_4b = pd.read_sql_query(query_7_4b, connection)


,profession,average_salary
0,"Маркетинговые, рекламные, BTL, дизайнерские, E...",350000.0
1,Лизинговые компании,300000.0
2,Кадровые агентства,237500.0
3,Издательская деятельность,236024.0
4,Консалтинговые услуги,222500.0
5,Разработка программного обеспечения,212308.0
6,"Системная интеграция, автоматизации технологи...",212150.0
7,"Управляющая, инвестиционная компания (управлен...",200000.0
8,"Интернет-компания (поисковики, платежные систе...",178740.0
9,Интернет-провайдер,175346.0


5а. Знание Microsoft Office по всем вакансиям

In [61]:
query_7_5a = f'''select count(v.id) as microsoft_office
                from vacancies v
                
                where (lower(v.key_skills) like '%ms%'
                or lower(v.key_skills) like '%microsoft%'
                or lower(v.key_skills) like '%excel%'
                or lower(v.key_skills) like '%powerpoint%'
                or lower(v.key_skills) like '%word%'
                or lower(v.key_skills) like '%access%'
                or lower(v.key_skills) like '%publisher%'
                or lower(v.key_skills) like '%office%')
            '''

In [62]:
# результат запроса
result_7_5a = pd.read_sql_query(query_7_5a, connection)
result_7_5a

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/2118278263.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7_5a = pd.read_sql_query(query_7_5a, connection)


,microsoft_office
0,8030


5б. Знание Microsoft Office по позициям, связанным с анализом данных

In [63]:
query_7_5b = f'''select count(v.id) as microsoft_office
                from vacancies v
                
                where (lower(v.name) like '%data scientist%'
                or lower(v.name) like '%data science%'
                or lower(v.name) like '%исследователь данных%'
                
                or (v.name like '%ML%'and v.name not like '%HTML%')
                
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%')
                
                and (lower(v.key_skills) like '%ms%'
                or lower(v.key_skills) like '%microsoft%'
                or lower(v.key_skills) like '%excel%'
                or lower(v.key_skills) like '%powerpoint%'
                or lower(v.key_skills) like '%word%'
                or lower(v.key_skills) like '%access%'
                or lower(v.key_skills) like '%publisher%'
                or lower(v.key_skills) like '%office%')
            '''

In [64]:
# результат запроса
result_7_5b = pd.read_sql_query(query_7_5b, connection)
result_7_5b

/var/folders/0j/pqz07rqd22q5j3qyrrpztqnr0000gn/T/ipykernel_71301/730949850.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7_5b = pd.read_sql_query(query_7_5b, connection)


,microsoft_office
0,46


**Выводы по дополнительному анализу**:

1.  Средняя зарплата в крупных городах показывает более подробную картину зарплатных ожиданий, чем зарплатная вилка (90т.) по всех регионам.
    - подтверждает необходимость более детального анализа, если, скажем, в определенном городе объявлена новая должность и необходима соответствующая корректировка заработной платы
2.  Анализ данных по большим компаниям, такие как: 'Ростелеком', 'Тинькофф', 'СБЕР', 'Газпром нефть' предпочитают рекламировать вакансии в крупных городах по сравнению с Яндексом.
    - анализ показывает, что предпочтения компании могут различаться в разных отраслях, нужно быть осторожным с предположениями по данным.
3.  Топ-10 самых высоких средних зарплат по регионам находятся за пределами России.
    - однако, но по позициям, связанным с анализом данных наблюдается лучший результат: позиции 5 по 10 занимают российские города; показывает растущий интерес к отрасли, и потенциальный будущий рост
4.  Cамые высокодоходные отрасли связаны с пищевой промышленностью: это логичная траектория, поскольку одна из самых основных биологических потребностей человека — это еда и питье;
    - позиции, связанные с анализом данных, в основном (по крайней мере по заработной плате) связаны со сферой услуг: маркетинг, реклама, подбор персонала, консалтинговым услугам
5.  Работодатели заинтересованы в знание Microsoft Office (Word, Excel, Powerpoint) - 16,3%, упоминается во всех вакансиях.
    - гораздо меньше по позициям, связанных с анализом данных (0,09%): либо навыки не столь необходимы по сравнению с языками программирования, как Python и SQL, либо навыки не указаны в самом объявлении, но знание навыка ожидается.

## Общий Вывод по Проекту

**Общие выводы**:

1. Набор данных содержит пять таблиц: четыре, из которых содержат информацию о вакансиях (49197 записей), работодателях (23501 записей), местоположении регионов (1362 записей), и сфере деятельности (294 записей); пятая таблица является связующей между таблицей работодателей и сфер деятельности.
2. Наибольшее количество вакансий сосредоточено в крупных городах и, похоже, благоприятствует частному сектору, и больше >50% вакансий не имеют информации о потенциальной заработное плате.
3. Предпочтение отдаётся полному рабочему дню и полному графику, с ростом интереса к дистанционной работе.
4. Предпочтение стажа работы начинается от 1-3 лет и 3-6 лет. Вакансии, требующие более длительного опыта работы (>6 лет), скорее всего, будут рекламироваться внутри компании, поэтому на HeadHunter будут рекламироваться реже.
5. Наибольшее количество вакансий рекламируют IT, банковский, и энергетические сектор, и поэтому те же отрасли рекламируют вакансии в большем количестве регионов.
6. Около 17% вакансий не имеют подробного описания по отрасли: либо из-за неправильного ввода данных, либо из-за того, что работодатели ищут более квалифицированных работников, которые могут уверенно решать различные типы задач.
7. Около 3,6% позиций по анализу данным, свидетельствуют о медленном росте отрасли.
8. Python (0,71%) более популярен, чем SQL (201, 0,4%): знание первого программного языка важнее для сферы анализа данных.
   - в знание Microsoft Office наблюдается гораздо меньшая заинтересованость: либо навыки не столь необходимы по сравнению с языками программирования,либо навыки не указаны в самом объявлении, но знание навыка ожидается
   - наличие разнообразный навыков, так же очень высоко: в среднем необходимо иметь около 6-7 навыков для работы в анализе данных
9. Заработная плата по позициям, связанным с анализом данных намного выше среднего по стране.
10. Высокая средняя заработная плата наблюдается за пределами России, однако, по позициям, связанным с анализом данных наблюдается лучший результат: позиции 5 по 10 занимают российские города; показывает растущий интерес к отрасли, и потенциальный будущий рост.

In [65]:
# закрываем соединение
connection.close()